Python has many libraries for **running** and **managing** child processes. This is why Python is a great language for gluing other tools together (ex: command line utilities). 

Python has had many ways to run subprocesses over the years:
* popen
* popen2
* os.exec

but now it's better to use
* subprocess

### subprocess 

Here is the *Popen* constructor starts the process. The *communicate* method reads the child process's output and waits for termination

In [1]:
import subprocess

In [2]:
proc = subprocess.Popen( ['echo', 'Hello from the child!' ], stdout = subprocess.PIPE)

In [3]:
out, err = proc.communicate()

In [4]:
print(out.decode('utf-8'))

Hello from the child!



#### example

In [20]:
proc1 = subprocess.Popen( ['echo', '$PYTHONPATH'] , stdout = subprocess.PIPE)

In [21]:
out, err = proc1.communicate()

In [25]:
print(out)
print(type(out))

b'$PYTHONPATH\n'
<class 'bytes'>


In [26]:
print(out.decode('utf-8'))

$PYTHONPATH



Child processes run independently from their parent process, the *Python* interpreter. Their status can be polled any time while Python does other work.

In [29]:
proc = subprocess.Popen(['sleep', '0.00001'])
while proc.poll() is None:
    print("Working ....")
print('Exit Status', proc.poll())

Working ....
Working ....
Working ....
Working ....
Exit Status 0


Decoupling the **child process** from the **parent process** means that the parent process is free to run many child processes in parallel. 
We can do this by starting all the child processes together upfront.

In [39]:
def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

In [40]:
import time

In [41]:
start = time.time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)

Later, we can wait for them to finish I/O and terminate with the *communicate* method

In [42]:
for proc in procs:
    proc.communicate()
end = time.time()
print("Finished in %.3f seconds" % (end - start))

Finished in 0.391 seconds


Nice tutorial [here](http://www.bogotobogo.com/python/python_subprocess_module.php)

##  Subprocess Module

This allows us to 
* spawn new processes
* connect to their input/output/error pipes
* obtain their return codes

They replace 
* os.system()
* os.spawn()
* os.popen()
* popen2.*()
* commands.*()

We can not run UNIX commands in python, to do so, we need to use subprocesses.

### os.system 

This is the simplest way to run UNIX commands.

In [43]:
import os
os.system('echo $PWD')

0

In [46]:
os.system('echo %s' %'$HOME')

0

NB: this work from the terminal, but does not give anything from a notebook!

The reason is that the *stdout* is the terminal !

In [53]:
!$PWD

/bin/sh: /Users/j35/git/notebooks/python_101/concurrency_and_parallelism: is a directory


Here is a way to make it work on a notebook, by redirecting it to a file. 

In [54]:
os.system('echo $HOME > outfile')

0

In [55]:
f = open('outfile','r')
f.read()

'/Users/j35\n'

### os.popen()

This open a pipe to and from command. The return value is an open file object connected to the pipe, which can be read or written depending on wheter mode is '**r**' (default) or '**w**'. 

In [56]:
stream = os.popen('echo $HOME')

In [57]:
stream.read()

'/Users/j35\n'

### subprocess.call()

This is like **popen**. But it simply wait until the command completes and gives us the return code.

example:
```
subprocess.call(args, *, stdin=None, stdout=None, stderr=None, shell=False)
```
This run the command described by *args*. Wait for the command to complete, then return the **returncode** attribute

In [63]:
import subprocess

In [64]:
subprocess.call(['ls', '-l'])

0

In [66]:
subprocess.call('echo $HOME', shell=True)  #necessary to avoid an OSError

0

### subprocess.check_call()

Like call but to check !

In [67]:
subprocess.check_call('false')

CalledProcessError: Command 'false' returned non-zero exit status 1

### subprocess.check_output()

To capture the output.

In [76]:
output = subprocess.check_output(['ls', '-l'],
                                 shell=True, 
                                 stdin=None, 
                                 stderr=None, 
                                 universal_newlines=True)

In [77]:
print(output)

index.ipynb
outfile
subprocess_manage_child_process.ipynb



### subprocess.Popen()

Process creation and management is done thanks to the **popen** class.

**subprocess.Popen** executes a child program in a new process. 

In [78]:
proc = subprocess.Popen(['echo','Hellow world!'],
                        stdout = subprocess.PIPE)

the **communicate()** returns a **tuple** (stdoutdata, stderrdata). If we don't include **stdout=subprocess.PIPE** or **stderr=subprocess.PIPE** in the Popen call, we will just get none back

In [79]:
stddata = proc.communicate()

In [80]:
stddata

(b'Hellow world!\n', None)

Script to test stdout and stderr (std_test.py)

In [82]:
proc = subprocess.Popen(['python','std_test.py'])
proc.communicate()

(None, None)

In [84]:
proc = subprocess.Popen(['python','std_test.py'],
                       stdout=subprocess.PIPE)
proc.communicate()

(b'Testing message to stdout\n', None)

In [85]:
proc = subprocess.Popen(['python','std_test.py'],
                       stderr=subprocess.PIPE)
proc.communicate()

(None, b'Testing messgae to stderr\n')

In [86]:
proc = subprocess.Popen(['python','std_test.py'],
                        stdout=subprocess.PIPE,
                       stderr=subprocess.PIPE)
proc.communicate()

(b'Testing message to stdout\n', b'Testing messgae to stderr\n')

if we want the **stderr** to be piped to **stdout**, we do

**stderr = subprocess.STDOUT**

In [87]:
proc = subprocess.Popen(['python','std_test.py'],
                        stdout = subprocess.PIPE,
                        stderr = subprocess.STDOUT)
proc.communicate()

(b'Testing messgae to stderr\nTesting message to stdout\n', None)

We do not have a **stderr** message anymore !

### subprocess.Popen() - stdin

It's possible to send data to the **stdin**

```
# write_to_stdin.py
import sys
input = sys.stdin.read()
sys.stdout.write('Received: %s'%input)
```

to send a message to **stdin**, we pass a string to send as an input argument to **communicate**

In [111]:
proc = subprocess.Popen(['python','write_to_stdin.py'], 
                        stdin = subprocess.PIPE,
                       stdout = subprocess.PIPE,
                       stderr = subprocess.PIPE)

because we are using python3, we need to use bytes instead

In [112]:
proc.communicate(bytes('Heloodfdfdf','UTF-8')) 

(b'Received: Heloodfdfdf', b'')

### subprocess.Popen() - shell pipeline

In [106]:
p1 = subprocess.Popen('set', stdout = subprocess.PIPE, shell = True)
p2 = subprocess.Popen(['grep', 'python'], stdout = subprocess.PIPE, shell = True)

In [107]:
p1.stdout.close()

In [108]:
output = p2.communicate()[0]

In [109]:
output

b''